In [34]:
import random
from collections import deque
import math
from ortools.linear_solver import pywraplp

#Initialisation des variables communes
tabId = []
tabProfit = []
tabPoids = []
nbItem = 0
maxCap = 0


In [35]:
#Remplissage des tableaux qui représente le graphe par lecture du fichier
def lireFichier(f):
    tabId.clear()
    tabProfit.clear()
    tabPoids.clear()

    fichier = open(f, "r")
    fichier.readline()
    fichier.readline()
    fichier.readline()
    nbItem = int((fichier.readline()).split(" ")[1])
    maxCap = int((fichier.readline()).split(" ")[1])
    fichier.readline()
    fichier.readline()
    for i in range(int(nbItem)):
        element = fichier.readline().strip().split(" ")
        tabId.append(int(element[0]))
        tabProfit.append(int(element[1]))
        tabPoids.append(int(element[2]))

    fichier.close()
    return int(nbItem), int(maxCap)


In [36]:
### Méthode du Tabou ### 
#Avec lenTabu : la longueur de la liste tabou et maxIter : le nombre d'itération
def tabuSearch(lenTabu = 1, maxIter = 500):
    #Création de la solution initial
    xmax = solutionInitial()
    x = xmax.copy()
    fmax = sommeProfit(xmax)
    queueTabu = deque([])
    
    #Boucle sur le nombre d'itération voulu
    for i in range (0,maxIter):
        #Récupération des voisins de la solution et de leur changement
        listVoisin, listChangement = listerVoisin(queueTabu, x)
        #Récupération du meilleurs voisins et du bit changé par celui-ci
        xbis, bitChanged = maximiserProfit(listVoisin, listChangement)
        #Ajout du bit changé dans la liste si le profit est plus faible que la solution actuelle
        if sommeProfit(xbis) < sommeProfit(x) :
            if(lenTabu != 0):
                if(len(queueTabu)==lenTabu):
                    queueTabu.popleft()
                queueTabu.append(bitChanged)
        #Mise à jour de la meilleurs solution si besoin
        if sommeProfit(xbis) > sommeProfit(xmax):
            xmax = xbis.copy()
            fmax = sommeProfit(xbis)
        x = xbis.copy()
    return xmax, fmax

#Calcule le profit d'une liste d'items
def sommeProfit(listItem):
    sum = 0
    for i in range(0, nbItem):
        if listItem[i] == 1 :
            sum += int(tabProfit[i])
    return sum

#Vérifie si une solution est accepté (si elle ne dépasse pas le poids max du sac)
def isAccepted(listItem):
    sum = 0
    i=0
    for i in range(0, nbItem):
        if listItem[i] == 1 :
            sum += int(tabPoids[i])
    return (sum <= maxCap)

#Liste tous les voisins possibles et leurs changements respectifs
def listerVoisin(tabTabu, listItem):
    listVoisin = []
    listChangement = []
    for i in range(0,nbItem):
        voisin = listItem.copy()
        #Change un bit de la solution donné
        if voisin[i] == 0 :
            voisin[i] = 1
        else :
            voisin[i] = 0
        #Vérifie si le voisin peut être accepté avant de l'ajouter
        if not i in tabTabu and isAccepted(voisin):
            listVoisin.append(voisin)
            listChangement.append(i)
    return listVoisin, listChangement

#Trouve le meilleur voisin de la liste et le retourne avec le bit changé dans celui-ci
def maximiserProfit(listVoisin, listChangement):
    max=-1
    betterVoisin = []
    bitChanged = -1
    for i in range(0, len(listVoisin)):
        if max < sommeProfit(listVoisin[i]) : 
            max = sommeProfit(listVoisin[i])
            betterVoisin = listVoisin[i]
            bitChanged = listChangement[i]
    return betterVoisin, bitChanged

#Solution initial avec aucun item dans le sac
def solutionInitial():
    return [0] * int(nbItem)

In [37]:
#Test
fichier_test = "data/pi-12-100-1000-001.kna"
nbItem, maxCap = lireFichier(fichier_test)
print(f"Capacité max: {maxCap}")
#Peut renseigner lenTabu et maxIter dans les paramètres de la fonction (sinon 1 et 500 par défaut)
best_solution, best_profit = tabuSearch()
print(f"Meilleure solution trouvée: {best_solution}")
print(f"Profit maximal obtenu: {best_profit}")

Capacité max: 970
Meilleure solution trouvée: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Profit maximal obtenu: 970


In [38]:
### Méthode du recuit simulé ###
# Paramètres :
# T       : Température initiale
# alpha   : Facteur de refroidissement (entre 0 et 1)
# Tmin    : Température minimale (seuil d'arrêt)
# iterMax : Nombre maximal d’itérations
def recuitSimule(T=100, alpha=0.99, Tmin=0.1, iterMax=1000):

    x = solutionInitial()  # Génère une solution initiale 
    x_best = x.copy()      # Meilleure solution rencontrée jusqu’à présent
    f_best = sommeProfit(x)  # Valeur du profit pour cette meilleure solution

    print("Début du recuit simulé...")
    print(f"Température initiale: {T}")
    
    for iter in range(iterMax):
        if T < Tmin:
            break  # Stoppe l'algorithme si la température est trop basse

        # Génère un voisin de la solution actuelle
        voisin = genererVoisin(x)
        deltaE = sommeProfit(voisin) - sommeProfit(x)  # Variation du profit

        # Accepte le voisin si meilleur, ou avec une proba qui dépend de la température
        if deltaE > 0 or random.random() < math.exp(deltaE / T):
            x = voisin.copy()  # Accepte le voisin
            # Met à jour la meilleure solution si le voisin est meilleur
            if sommeProfit(x) > f_best:
                x_best = x.copy()
                f_best = sommeProfit(x)

        T *= alpha  # Refroidissement : on diminue progressivement la température

        # Affichage toutes les 100 itérations pour le suivi
        if iter % 100 == 0:
            print(f"Iteration {iter}, Température: {T:.2f}, Profit actuel: {sommeProfit(x)}")

    print("Recuit simulé terminé!")
    return x_best, f_best


# Fonction pour générer un voisin d’une solution donnée
# Elle change un seul item aléatoirement (flip 0 <-> 1), et retourne la solution si elle est valide
def genererVoisin(solution):
    voisin = solution.copy()
    i = random.randint(0, nbItem - 1)  # Choisit une position aléatoire à modifier
    voisin[i] = 1 - voisin[i]          # Inverse la valeur (0 -> 1 ou 1 -> 0)
    return voisin if isAccepted(voisin) else solution  # Retourne la nouvelle solution si elle respecte les contraintes


# Génère une solution initiale : ici, un sac à dos vide 
def solutionInitial():
    return [0] * nbItem



In [39]:
# test
fichier_test = "data/pi-12-100-1000-001.kna"
nbItem, maxCap = lireFichier(fichier_test)
print(f"Capacité max: {maxCap}")
best_solution, best_profit = recuitSimule()
print(f"Meilleure solution trouvée: {best_solution}")
print(f"Profit maximal obtenu: {best_profit}")

Capacité max: 970
Début du recuit simulé...
Température initiale: 100
Iteration 0, Température: 99.00, Profit actuel: 52
Iteration 100, Température: 36.24, Profit actuel: 766
Iteration 200, Température: 13.26, Profit actuel: 811
Iteration 300, Température: 4.86, Profit actuel: 811
Iteration 400, Température: 1.78, Profit actuel: 811
Iteration 500, Température: 0.65, Profit actuel: 811
Iteration 600, Température: 0.24, Profit actuel: 811
Recuit simulé terminé!
Meilleure solution trouvée: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Profit maximal obtenu: 811


In [40]:
### Méthode génétique ### 
#Avec probaCross : la probabilté de croisement, nbBest : le nombre d'individue reproduit par élitisme, 
# nbGen : le nombre de génération et nbPop : le nombre d'individu dans la population
def genetique(probaCross = 0.5, nbBest = 20, nbGen = 100, nbPop = 100):
    #Génération de la population de départ
    population = genPopulationInitial(nbPop)
    #Calcule de la meilleurs solution dans la population
    bestKnown = caculerBest(population)
    #Boucle sur le nombre de génération voulu
    for i in range(nbGen):
        #Création de la population par roulette biaisé
        popRoulette = reproductionRoulette(population)
        #Reproduction de nbBest individue par stratégie élitiste
        nextPopulation = reproductionBest(nbBest, population)
        for j in range(nbBest + 1, nbPop):
            #Choix entre un croisement ou une mutation
            if random.random() < probaCross :
                nextPopulation.append(croisement(popRoulette))
            else : 
                nextPopulation.append(mutation(popRoulette))
        population = nextPopulation.copy()
        #recalcule de la meilleurs solution avec la nouvelle population
        bestKnown = caculerBest(population)
    return bestKnown, sommeProfit(bestKnown)

#Génération d'une population de départ
def genPopulationInitial(nbIndividus):
    population = []
    #On veut remplir au maximum 70% du sac pour ne pas tendre vers un maximum trop vite
    capCible = 0.7 * maxCap

    for j in range(nbIndividus):
        #Initalise la solution à 0
        solution = [0] * nbItem
        poidsTotal = 0
        indices = list(range(nbItem))
        random.shuffle(indices)
        #Ajout d'items au hasard dans le sac jusqu'au remplissage voulu
        for i in indices:
            poidsObjet = int(tabPoids[i])
            if poidsTotal + poidsObjet <= capCible:
                solution[i] = 1
                poidsTotal += poidsObjet

        population.append(solution)

    return population

#Retourne la meilleurs solution dans la population donnée
def caculerBest(population):
    max=-1
    betterKnown = []
    for i in range(0, len(population)):
        if max < sommeProfit(population[i]) : 
            max = sommeProfit(population[i])
            betterKnown = population[i]
    return betterKnown

#Reproduit une population à l'aide d'une roulette biaisé
def reproductionRoulette(population):
    repro = []
    fitness = []
    #Calcule des fitness des individus de la population
    for ind in population:
        fitness.append(sommeProfit(ind))
    total = sum(fitness)
    
    #Calcule de la probabitlité cumulé des individues comparé au total
    probaCumul = []
    cumul = 0
    for profit in fitness:
        cumul += profit / total
        probaCumul.append(cumul)

    #Choix d'un individu en fonction de sa probabilité et d'un nombre aléatoire
    for i in range(0, len(population)):
        rand = random.random()
        for j in range(len(probaCumul)):
            if (rand <= probaCumul[j]):
                repro.append(population[j])
                break

    return repro

#Retourne les nbBest premières solutions de la population
def reproductionBest(nbBest,population):
    populationTrie = sorted(population, key=lambda ind: sommeProfit(ind), reverse=True)
    return populationTrie[0:nbBest]

#Retourne un croisement au hasard dans une population
def croisement(population):
    #Prend deux individus au hasard
    ind1 = population[random.randint(0, len(population) - 1)]
    ind2 = population[random.randint(0, len(population) - 1)]
    
    #Prend un point de coupe au hasard
    rand = random.randint(0, nbItem - 1)
    
    #Effectue le croisement
    crois = ind1[:rand] + ind2[rand:]
    #Si le croisement n'est pas une bonne solution, le transforme en bonne solution
    if (not isAccepted(crois)):
        crois = buildValidSolution(crois)
    return crois

#Retourne la mutation d'un individu au hasard
def mutation(population):
    #Prend un individu au hasard
    ind = population[random.randint(0, len(population) - 1)].copy()
    #Inverse un bit au hasard
    rand = random.randint(0, nbItem - 1)
    ind[rand] = 1 - ind[rand]
    #Si la mutation n'est pas une bonne solution, la transforme en bonne solution
    if (not isAccepted(ind)):
        ind = buildValidSolution(ind)
    return ind

#Transforme une mauvaise solution en solution accepté
def buildValidSolution(solution):
    nouvelle_solution = solution.copy()
    bit1 = []
    #Récupère tous les indexs des items dans le sac
    for j in range(nbItem) :
        if (solution[j] == 1):
            bit1.append(j)
    #Enlève un des objets au hasard jusqu'à que la solution soit bonne
    while not isAccepted(nouvelle_solution):
        rand = random.randint(0, len(bit1) - 1)
        i = bit1.pop(rand)
        nouvelle_solution[i] = 0

    return nouvelle_solution

In [41]:
#Test
fichier_test = "data/pi-12-100-1000-001.kna"
nbItem, maxCap = lireFichier(fichier_test)
print(f"Capacité max: {maxCap}")
#Peut renseigner probaCross, nbBest, nbGen et nbPop dans les paramètres de la fonction (sinon 0.5, 20, 100, 100 par défaut)
best_solution, best_profit = genetique()
print(f"Meilleure solution trouvée: {best_solution}")
print(f"Profit maximal obtenu: {best_profit}")

Capacité max: 970
Meilleure solution trouvée: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Profit maximal obtenu: 970


In [42]:
def solveurLineaire():
    solver = pywraplp.Solver.CreateSolver("CBC")
    if not solver:
        print("Le solveur n'a pas pu être créé.")
        return

    n = len(tabPoids)
    x = [solver.IntVar(0, 1, f"x_{i}") for i in range(n)]

    #Contrainte de capacité
    solver.Add(solver.Sum(tabPoids[i] * x[i] for i in range(n)) <= maxCap)

    #Fonction objectif : maximiser le profit
    solver.Maximize(solver.Sum(tabProfit[i] * x[i] for i in range(n)))
    solver.SetTimeLimit(10000)
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print("Solution trouvée :")
        for i in range(n):
            print(f"x_{i+1} = {x[i].solution_value()}")
        print(f"Profit maximal obtenu: {solver.Objective().Value()}")
    else:
        print("Aucune solution trouvée.")

In [43]:
#Test
fichier_test = "data/pi-12-100-1000-001.kna"
nbItem, maxCap = lireFichier(fichier_test)
print(f"Capacité max: {maxCap}")
solveurLineaire()
print(f"Meilleure solution trouvée: {best_solution}")
print(f"Profit maximal obtenu: {best_profit}")

Capacité max: 970
Solution trouvée :
x_1 = 0.0
x_2 = 0.0
x_3 = 0.0
x_4 = 0.0
x_5 = 0.0
x_6 = 0.0
x_7 = 0.0
x_8 = 0.0
x_9 = 0.0
x_10 = 0.0
x_11 = 0.0
x_12 = 0.0
x_13 = 0.0
x_14 = 0.0
x_15 = 0.0
x_16 = 0.0
x_17 = 0.0
x_18 = 0.0
x_19 = 0.0
x_20 = 0.0
x_21 = 0.0
x_22 = 0.0
x_23 = 0.0
x_24 = 0.0
x_25 = 0.0
x_26 = 0.0
x_27 = 0.0
x_28 = 0.0
x_29 = 0.0
x_30 = 0.0
x_31 = 0.0
x_32 = 0.0
x_33 = 0.0
x_34 = 0.0
x_35 = 0.0
x_36 = 0.0
x_37 = 0.0
x_38 = 0.0
x_39 = 0.0
x_40 = 0.0
x_41 = 0.0
x_42 = 0.0
x_43 = 0.0
x_44 = 0.0
x_45 = 0.0
x_46 = 0.0
x_47 = 0.0
x_48 = 0.0
x_49 = 0.0
x_50 = 0.0
x_51 = 0.0
x_52 = 0.0
x_53 = 1.0
x_54 = 0.0
x_55 = 0.0
x_56 = 0.0
x_57 = 0.0
x_58 = 0.0
x_59 = 0.0
x_60 = 0.0
x_61 = 0.0
x_62 = 0.0
x_63 = 0.0
x_64 = 0.0
x_65 = 0.0
x_66 = 0.0
x_67 = 0.0
x_68 = 0.0
x_69 = 0.0
x_70 = 0.0
x_71 = 0.0
x_72 = 0.0
x_73 = 0.0
x_74 = 0.0
x_75 = 1.0
x_76 = 0.0
x_77 = 0.0
x_78 = 0.0
x_79 = 0.0
x_80 = 0.0
x_81 = 0.0
x_82 = 0.0
x_83 = 0.0
x_84 = 0.0
x_85 = 0.0
x_86 = 0.0
x_87 = 0.0
x_88 = 0.0
x_89